In [1]:
import os
from pathlib import Path
import torch 
import wandb

CONFIG = dict (
    project = "hlc-polyp-detection",
    architecture = "fasterrcnn_resnext50_32x4d",
    dataset_id = "hlc-custom-polyp-detection",
    infra = "osx",
    num_classes = 2,
    max_epochs = 100,
    lr=0.01,
    min_lr=0.0000001,
    epochs=15,
    batch_size=4,
    nesterov=True,
    momentum=0.9,
    weight_decay=0.0005,
    clip_limit=0.25,
    difference=False,
    name="fancy_walrus"
)

ROOT_DIR = os.path.abspath("./")
DATA_DIR = os.path.join(ROOT_DIR, "data")
MODEL_DIR = os.path.join(ROOT_DIR, "model")
LOG_DIR = os.path.join(ROOT_DIR, "log")

NUM_CLASSES = 2

BATCH_SIZE = 4
INPUT_SIZE = 1024
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_WORKERS = 8

wandb_key = Path(os.path.join(ROOT_DIR, "wandb.txt")).read_text().strip()
os.environ["WANDB_API_KEY"] = wandb_key
os.environ["WANDB_NOTEBOOK_NAME"] = os.path.join(ROOT_DIR, "LightningFastRCNN-polyps.ipynb")

In [5]:
from src.PolypsPLDataModule import PolypsPLDataModule
from src.LightningFasterModule import LightningFasterModule
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

# Initialize the data module
polyp_dm = PolypsPLDataModule(data_dir='./data', batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

# Load the model from the checkpoint
model = LightningFasterModule.load_from_checkpoint(os.path.join(MODEL_DIR, "peppered_jaguar-chkpt-full.ckpt"))

# Initialize the WandB logger
wandb_logger = WandbLogger(project=CONFIG['project'], log_model="all")

# Initialize the trainer
trainer = Trainer(
    logger=wandb_logger,
    accelerator='gpu' if torch.cuda.is_available() else None,
    log_every_n_steps=1
)

# Run the test set
# trainer.test(model, datamodule=polyp_dm)

/Users/jarret/opt/anaconda3/envs/bolt-hlc/lib/python3.10/site-packages/pytorch_lightning/utilities/migration/utils.py:49: PossibleUserWarning: The loaded checkpoint was produced with Lightning v2.0.2, which is newer than your current Lightning version: v1.9.3
  rank_zero_warn(
wandb: Currently logged in as: spinonoir (hidden-layer-cake). Use `wandb login --relogin` to force relogin


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/jarret/opt/anaconda3/envs/bolt-hlc/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(


In [20]:
from src.PolypsPLDataModule import PolypsPLDataModule
from src.LightningFasterModule import LightningFasterModule
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger

# Initialize the data module
polyp_dm = PolypsPLDataModule(data_dir='./data', batch_size=BATCH_SIZE, num_workers=NUM_WORKERS)

# Load the model from the checkpoint
model = LightningFasterModule.load_from_checkpoint(os.path.join(MODEL_DIR, "lazy_panda-chkpt-full.ckpt"))

# Initialize the WandB logger
wandb_logger = WandbLogger(project=CONFIG['project'], log_model="all")

# Initialize the trainer
trainer = Trainer(
    logger=wandb_logger,
    accelerator='gpu' if torch.cuda.is_available() else None,
    log_every_n_steps=1
)

# Run the test set
# trainer.test(model, datamodule=polyp_dm)





/usr/local/lib/python3.9/dist-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │     0.598007082939148     │
│        test_avg_f1        │    0.6866459846496582     │
│    test_avg_precision     │    0.7771739363670349     │
│      test_avg_recall      │    0.6622670888900757     │
│          test_f1          │    0.6866463422775269     │
│        test_max_f1        │            1.0            │
│    test_max_precision     │            1.0            │
│      test_max_recall      │            1.0            │
│        test_min_f1        │            0.0            │
│    test_min_precision     │            0.0            │
│      test_min_recall      │            0.0            │
│      test_precision       │    0.7771736979484558     │
│        test_recall        │    0.6622670888900757     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.598007082939148,
  'test_precision': 0.7771736979484558,
  'test_recall': 0.6622670888900757,
  'test_f1': 0.6866463422775269,
  'test_max_precision': 1.0,
  'test_max_recall': 1.0,
  'test_max_f1': 1.0,
  'test_min_precision': 0.0,
  'test_min_recall': 0.0,
  'test_min_f1': 0.0,
  'test_avg_precision': 0.7771739363670349,
  'test_avg_recall': 0.6622670888900757,
  'test_avg_f1': 0.6866459846496582}]

In [6]:
import pytorch_lightning as pl

In [11]:
mos_sum = pl.utilities.model_summary.summarize(model, max_depth=4)

In [16]:
mos_sum

   | Name                                       | Type                     | Params
-----------------------------------------------------------------------------------------
0  | detector                                   | FasterRCNN               | 43.3 M
1  | detector.transform                         | GeneralizedRCNNTransform | 0     
2  | detector.backbone                          | BackboneWithFPN          | 26.9 M
3  | detector.backbone.body                     | IntermediateLayerGetter  | 23.5 M
4  | detector.backbone.body.conv1               | Conv2d                   | 9.4 K 
5  | detector.backbone.body.bn1                 | BatchNorm2d              | 128   
6  | detector.backbone.body.relu                | ReLU                     | 0     
7  | detector.backbone.body.maxpool             | MaxPool2d                | 0     
8  | detector.backbone.body.layer1              | Sequential               | 215 K 
9  | detector.backbone.body.layer2              | Sequential          

In [12]:

# write mos to file
with open(os.path.join(MODEL_DIR, "model_summary.txt"), "w") as f:
    f.write(mos_sum)

TypeError: write() argument must be str, not ModelSummary